# Analyzing the Analytics Job Market
                    
Group 4:
    Boer Feng, Di Niu, Kamran Qureshi, Naveen Ithikkat, Sifan Du
    
Develop an interactive visualization to answer these questions from jobs website data:
- How many jobs related to Analytics are posted in total?
- How many jobs in each city or state?
- Which are the companies having most job postings?
- Which are the skills most in demand?



### Scraping indeed.com

We have made a comparsion of several job posting websities, such as Glassdoor and linkedin. Glassdorr has limitied API and requires a user to sign in for complete access. Therefore, we settled with Indeed.com which is easy to access that information related to job postings. 

First of all, we import required packages that we need for scraping

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import string
import csv
import pandas as pd
from geopy.geocoders import Nominatim
import geopy
import zipcode
import numpy as np

The information that we plan to scrape from indeed. com are job titles, company names, salary, location (city and states), day when the job is posted and summary of the jobs. We used the package called beautifulsoup to acheive this. 

Since some of the job postings do not include all the information that we plan to scrape, such as salary, we came up with an if-else loop to return those missing valeus as N.A. in the csv file.  

In [2]:
finalList = []

statelist = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY',
             'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH',
             'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

for state in statelist:
    for i in range(0, 500, 10):
        # loop to iterate over all search pages
        url = "http://www.indeed.com/jobs?q=data+scientist&l=" + state + "&start=" + str(i)
        r = requests.get(url)
        data = r.text
        soup = bs(data, "html.parser")
        allJobs = soup.findAll('div', {'data-tn-component': 'organicJob'})

        for job in allJobs:
            templist = []
            jobTitle = job.find('a', {'class': 'turnstileLink'}).getText()
            # print(jobTitle.getText())
            ##print(resturant_name.getText())
            templist.append(jobTitle)
            
            if len(job.findAll('span', {'class': 'company'})) == 0:
                company = 'NA'
            else:
                company = job.find('span', {'class': 'company'}).find('span').getText().replace('\n', ' ').strip()
            templist.append(company)
            
            if len(job.findAll('nobr')) == 0:
                salary = 'NA'
            else:
                salary = job.find('nobr').getText()
            templist.append(salary)
            
            location = job.find('span', {'class': 'location'}).find('span').getText().replace('\n', ' ').strip()
            
            city = location[:location.find(',')]
            templist.append(city)
            
            state = location[location.find(',') + 2:location.find(',') + 4]
            templist.append(state)
            
            datePosted = job.find('span', {'class': 'date'}).getText().replace('\n', ' ').strip()
            templist.append(datePosted)
            
            summary = job.find('span',{'class': 'summary'}).getText().replace('\n',' ').strip()
            templist.append(summary)
            
            finalList.append(templist)


The list of jobs we get 

In [6]:
print(finalList)

[['QUANTITATIVE BUSINESS ANALYST USA (AL)', 'managementsolutions', 'NA', 'Birmingham', 'AL', '9 days ago', 'Statistical treatment of data (data mining). Knowledge of SAS, R, Python and/or SQL programming. Machine learning and statistical modelling skills:....'], ['Modeling, Simulation & Analysis - Systems Engineer I', 'Raytheon', 'NA', 'Huntsville', 'AL', '22 days ago', 'The Modeling Simulation and Analysis Center within Raytheon Missile Systems (RMS) consists of a highly skilled group of scientists and engineers developing the...'], ['Statistician', 'SunGard', 'NA', 'Birmingham', 'AL', '26 days ago', 'Position Responsibilities\\Requirements Job Duties Find ways of improving processes in and outside of the project in coordination with customer Work with prime...'], ['Quantitative Models Analyst', 'CADENCE BANK, N.A.', 'NA', 'Birmingham', 'AL', '22 days ago', 'Advanced skills with database development, maintenance, and extraction of data for reporting. Knowledge with SQL, SAP Business O

After getting those job information from Indeed.com. We used pandas to create a data frame and sent that data to a csv file. Now a csv file named "jobs" is created and contains all the information that we desires and ready to be cleaned and then analyzed. 

In [3]:
heading=["job_title","company","salary","city","state","day_posted","summary"]
finalData = pd.DataFrame(finalList,columns = heading)
finalData = finalData.loc[finalData['state'].isin(statelist)]
finalData.to_csv('jobs.csv',index=False)

### Finding co-ordinates of cities 

Function that uses geopy package to get latitude and longitude of cities. For plotting job listings on a map, we need co-ordinates of cities for which we first define a function that takes city and state as input and returns latitude and longitude. Then, we pull unique city and state combinations and pass those to the function. The resulting dataframe has city state latitude and longitude information which is sent to a csv. The resulting csv is later used in shiny to plot jobs on a map.

In [4]:
def do_geocode(address):
    try:
        return geolocator.geocode(address,addressdetails=True, timeout=None)        
    except geopy.exc.GeocoderTimedOut:
        return do_geocode(address)

Pulling unique cities and states from the above obtained data and obtaining their co-ordinates using geopy

In [7]:
geolocator=Nominatim()
dc = finalData[['city','state']]
dc = dc.drop_duplicates()
dc['address'] = dc['city'] + ' ' + dc['state']
for row in dc['address']:
    coord = do_geocode(row)
    if coord == None:
        dc.loc[dc.address == row, 'latitude'] = 'NA'
        dc.loc[dc.address == row, 'longitude'] = 'NA'
        #print(dc)
    else:
        dc.loc[dc.address == row, 'latitude'] = coord.latitude
        dc.loc[dc.address == row, 'longitude'] = coord.longitude



In [9]:
dc.to_csv('cod.csv',index=False)